In [ ]:
# #Extracting the stored dataframes from the previous notebook using the pickle module
# import pandas as pd
# consulting_club_posts = pd.read_pickle('consulting_club_posts.pkl')
# comments_df = pd.read_pickle('comments_df.pkl')

In [7]:
import ollama
# embed_model = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'    Can't take large enough data
#Use the command olllama serve in the terminal to start the ollama server
embed_model = 'hf.co/bartowski/granite-embedding-30m-english-GGUF'
lang_model = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

ollama.pull(embed_model)
ollama.pull(lang_model)

consulting_club_posts['embedding'] = consulting_club_posts['Post Text'].apply(
    lambda t: ollama.embed(model=embed_model, input=t)['embeddings'])

comments_df['embedding'] = comments_df['Comment'].apply(
    lambda t: ollama.embed(model=embed_model, input=t)['embeddings'])

# Formats it so each element is not a list of a list unnecessarily
consulting_club_posts['embedding'] = consulting_club_posts['embedding'].apply(lambda x: x[0] if len(x) == 1 else x)
comments_df['embedding'] = comments_df['embedding'].apply(lambda x: x[0] if len(x) == 1 else x)

In [8]:
# Creating weights for each embedding based on score values, plugging them into a tuned sigmoid function
# 0-200 has factor ~1, 200-500 has factor ~1.3, 500-700 has factor ~1.6, 700+ has factor ~1.8
# Doesn't function well, need to include comment score scaling as well if implemented

# consulting_club_posts['Score Factor'] = consulting_club_posts['Score'].apply(lambda x: 1 + 1/(1+20*pow(2, -x/100)))

# consulting_club_posts['Weighted embedding'] = consulting_club_posts['embedding'].combine(consulting_club_posts['Score Factor'], lambda lst, weight: [weight * x for x in lst])
# print(consulting_club_posts['Weighted embedding'], consulting_club_posts['Score'])

In [9]:
def cosine_similarity(a, b):
    dot_prod = sum([x * y for x, y in zip(a, b)])
    mag_a = pow(sum([pow(x, 2) for x in a]), 0.5)
    mag_b = pow(sum([pow(y, 2) for y in b]), 0.5)
    return mag_a * mag_b and (dot_prod) / (mag_a * mag_b)   # add mag_b to dot_prod for weightage

In [10]:
def retrieve_data(query, n=5):
    query_embed = ollama.embed(model=embed_model, input=query)['embeddings']
    similarities = consulting_club_posts['embedding'].apply(lambda x: cosine_similarity(query_embed[0], x))     # Change to 'weighted embedding' for weightage
    pd.concat([similarities, comments_df['embedding'].apply(lambda x: cosine_similarity(query_embed[0], x))])   
    return similarities.nlargest(n)

In [11]:
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve_data(input_query)


instruction_prompt = f'''You are a helpful chatbot aimed to help UC Berkeley students learn about and choose clubs to join.
Use only the following pieces of context to answer the question. Don't make up any new information:
{'\n'.join([f' - {consulting_club_posts.loc[i, 'Post Text']}' for i in retrieved_knowledge.index])}
'''

print(instruction_prompt)

You are a helpful chatbot aimed to help UC Berkeley students learn about and choose clubs to join.
Use only the following pieces of context to answer the question. Don't make up any new information:
 - I've seen a lot of reddit posts about consulting clubs at berkeley but I think it doesn't make sense to rank them by acceptance rate. For all the incoming freshmen, this is THE ultimate guide based on funding, prestige, alumni network, and opportunities. Feel free to fight me in the comments on this.
 - I am a freshman at Berkeley and wanna recruit for investment banking/consulting. There are so many clubs on campus and esp becuz first two weeks are remote I really don’t know what to choose from. 
Will appreciate any insights on rankings/differences among these clubs like BC, Voyager v.s. CBA, UFA, and BAP etc. I know business clubs here are very cutthroat so any advice would be appreciated.
 - I'm a freshman this year at Berkeley and was just wondering which consulting clubs are known t

In [12]:
stream = ollama.chat(
  model=lang_model,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

# print the response from the chatbot in real-time
print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)


Chatbot response:
The best coding club at UC Berkeley can vary depending on your interests, strengths, and preferences. Here are some of the top coding clubs at UC Berkeley:

1. **Net Impact**: While Net Impact is known for its emphasis on social entrepreneurship and impact investing, it also has a strong focus on technology innovation. Members have access to resources and networks that support their tech-related goals.
2. **BC Consulting**: BC Consulting is a well-established consulting club that offers training programs in areas like data analysis, machine learning, and digital marketing. Members have opportunities to work with real clients and build professional experience.
3. **Voyager** (formerly known as **UC Berkeley Consulting Club**): Voyager focuses on providing technical support for non-profit organizations, startups, and small businesses. Members have the opportunity to work on projects that benefit social or environmental causes.
4. **SAP Ecosystem**: SAP is a global leade